In [64]:
import sys;sys.path.append('..')
from ppanlp import *
ppa = PPA()

In [65]:
# ppa.ner_parse(min_doc_len=10,max_per_cluster=200)

In [71]:
from string import punctuation


class NERModel:
    def __init__(self, corpus=None):
        self.corpus = corpus if corpus is not None else PPA()
        self.ent2pages=defaultdict(set)
        self.ent2count=Counter()
        
    def iter_ents(self,ent_types:set=None,lim=None):
        with self.corpus.ents_db(flag='r') as db:
            total=len(db)
            iterr=tqdm(db.items(),desc='Iterating over saved ents',position=0,total=total)
            for page_id,ents in iterlim(iterr,lim):
                for ent,ent_type in ents:
                    if not ent_types or ent_type in ent_types:
                        ent = ent.strip(punctuation).title()
                        yield page_id,ent,ent_type

    def iter_persons(self, **kwargs):
        kwargs['ent_types']={'PERSON'}
        yield from self.iter_ents(**kwargs)
    
    def count_ents(self, **kwargs):
        self.ent2count=Counter()
        for page_id,ent,ent_type in self.iter_ents(**kwargs):
            self.ent2pages[ent].add(page_id)
            self.ent2count[ent]+=1
        self.ent2count_s = pd.Series(self.ent2count).sort_values(ascending=False)
        return self.ent2count_s
    
    def count_persons(self, **kwargs):
        kwargs['ent_types']={'PERSON'}
        return self.count_ents(**kwargs)


In [72]:
ner = NERModel()

In [74]:
ner.count_persons(lim=None)

Iterating over saved ents: 100%|██████████| 526697/526697 [01:42<00:00, 5139.83it/s] 


John              18502
Milton            15656
Chaucer           15409
Shakespeare       13490
Pope              10932
                  ...  
Gi'R                  1
Lifh-Mung- Gur        1
La'.Italle            1
Ti'Hi                 1
Babauld               1
Length: 831153, dtype: int64

In [75]:
s = ner.ent2count_s
s[s>1]

John                      18502
Milton                    15656
Chaucer                   15409
Shakespeare               13490
Pope                      10932
                          ...  
Old Ironsides Holmes          2
A Butterfly Wordsworth        2
Poetry Whittier               2
Ilkewife                      2
Jehovrah                      2
Length: 171982, dtype: int64

In [86]:
df = pd.DataFrame({'count':s}).rename_axis('name')
df['is_valid'] = ''
df

,count,is_valid
name,,
John,18502,
Milton,15656,
Chaucer,15409,
Shakespeare,13490,
Pope,10932,
...,...,...
Gi'R,1,
Lifh-Mung- Gur,1,
La'.Italle,1,


In [88]:
ofn=os.path.join(ppa.path_data, 'data.ner.to_anno.csv')
df[df['count']>=100].to_csv(ofn)